In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/checknft_collections.csv').iloc[:100000]

In [3]:
full_names_ds = df['name'].dropna().to_list()

In [4]:
from datasketch import MinHashLSHForest, MinHash
from multiprocessing import Pool


def get_minhash(name: str) -> MinHash:
    m_hash = MinHash(num_perm=128)
    for d in name.split():
        m_hash.update(d.encode('utf8'))
    return m_hash

forest = MinHashLSHForest(num_perm=128)


    

    


In [ ]:
from tqdm.notebook import tqdm_notebook

with Pool() as pool:
    hash_values = list(tqdm_notebook(pool.map(get_minhash, full_names_ds),
                            total=len(full_names_ds)))

Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/work/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/work/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/work/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/work/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'get_minhash' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/work/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/work/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/work/lib/python3

In [ ]:
for name, hash_value in zip(full_names_ds, hash_values):
    forest.add(name, hash_value)


In [ ]:
forest.index()